In [1]:
import nest_asyncio
import aiohttp
import asyncio
import nest_asyncio
import re
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
# import google.generativeai as genai
import time
import requests
import ast
import json
from sklearn.metrics.pairwise import cosine_similarity
import re
from llms import chatGPT
from llms import gemini


x_chat = chatGPT()
x_gemini = gemini()

In [2]:
df = pd.read_csv("../output/data/data_post_chat_gpt.csv")
df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>='11/30/2022']

In [2]:
generated_description_prompt = """
Your role is to describe $company ($website)'s product. 
Then, provide a confidence interval on scale on 1-10 on how sure you are about the response. Please be reasonable.

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

EXAMPLES BELOW
************************
Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo to generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution  providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.
************************

YOUR TURN:
Company: $company
Wesbite: $website
Current Description: $description
________________________
Description (two sentences):
Confidence Interval:
Reasoning:
________________________

"""



In [3]:
parsed_description_prompt = """
Your role is to describe what jobs/tasks, industries, and customer that $company is targeting. Then, provide a confidence interval (1-10) on scale on 1-10 on how sure you are about the response. Please be reasonable.

A response should include:
- tasks/jobs being automated
- the industry that the startup applies to
- specific customers using the tool. DO NOT INCLUDE AN EXPLANATION

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

EXAMPLES BELOW
************************
Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Vetinarians, Pet Businesses

Company: Thunder
Wesbite: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks 
Industry: Cloud Computing
Customers: Developers, GPU Owners

Company: NonprofitsHQ
Websit: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations
************************

YOUR TURN:
________________________
Company: $company
Wesbite: $website
Description: $description
________________________
Tasks/Jobs (comma separated list of 4, short):
Industry (1 item):
Customers (comma separated list): 
________________________
Confidence Interval:
Reasoning:
________________________
"""


In [4]:
examples_prompt = """
Your role is to provide 3 two-sentence examples of how the product from $company might be used. Do not mention the name of the company in the examples, and keep the descriptions broad.

Each example should include:
- A detailed description of the job that the tool automates and who performs that job and when.
- The ONET job being automated (preferably one from the database https://www.onetonline.org/) and the ONET task that the tool replaces (preferably one that from https://www.onetcenter.org/dictionary/20.1/excel/task_statements.html, include task id)
- A confidence interval (1-10) indicating how sure you are about the accuracy of your response.

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

The goal is to map each example back to ONET jobs. If the job being automated is recognized by ONET, please use the ONET job title. 
If it is not typically found in ONET, use best judgement!

EXAMPLES BELOW: 
************************
Company: Blanc
Website: tryblanc.ai
Blanc is a compliance automation platform designed for fintech companies. It helps streamline regulatory compliance processes by providing a centralized hub for managing policies, monitoring activities, and generating reports.
_________________________
Example 1: A compliance officer at a fintech company uses Blanc to automate the process of creating and updating compliance policies, ensuring all documents are current and accessible to relevant team members.
ONET JOB automated 1: Compliance Officers that verify that all firm and regulatory policies and procedures have been documented, implemented, and communicated.
ONET JOB 1: Compliance Officers
_________________________
Example 2: A fintech company uses Blanc to generate automated compliance reports for regulatory audits, ensuring all necessary documentation is readily available and organized.
ONET JOB automated 2: Compliance Officers Prepare reports of activities, evaluations, recommendations, or decisions.
ONET JOB 2: Compliance Officers
_________________________
Example 3: A fintech company uses Blanc to monitor real-time transactions for potential compliance violations, triggering alerts and generating reports for further investigation.
ONET JOB automated 3: Compliance Officers that identify compliance issues that require follow-up or investigation.
ONET JOB 3: Compliance Officers



Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
________________
Example 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
ONET JOB automated 1: Solar Photovoltaic Installers that diagram layouts and locations for photovoltaic (PV) arrays and equipment, including existing building or site features.
ONET JOB 1: Solar Photovoltaic Installers
________________
Example 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
ONET JOB automated 2: Solar Photovoltaic Installers that prepare solar installation project proposals, quotes, budgets, or schedules.
ONET JOB 2: Solar Photovoltaic Installers
________________
Example 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
ONET JOB automated 3:  Solar Engery Installation Managers that monitor work of contractors and subcontractors to ensure projects conform to plans, specifications, schedules, or budgets.
ONET JOB 3: Solar Energy Installation Managers
________________
************************

YOUR TURN:
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
Example 1: 
ONET JOB automated 1:
ONET JOB 1: 
Confidence Interval 1:
Reasoning 1:
________________
Example 2: 
ONET JOB automated 2:
ONET JOB 2: 
Confidence Interval 2:
Reasoning 2:
________________
Example 3:
ONET JOB automated 3:
ONET JOB 3: 
Confidence Interval 3:
Reasoning 3:
________________
"""


In [31]:
# Apply nest_asyncio
nest_asyncio.apply()


class prompting():
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["organization name", "value"])

    def set_current_results_df(self, results_df):
        self.results_df = results_df

    async def iterate(self, df, prompt_template, args, value, batch_size=10, start=0, end=False):
        if end == False:
            end = len(df)
        self.results_df = pd.DataFrame(columns=["organization name", value])
        if value in list(df.columns):
            if start != 0:
                self.results_df = pd.concat([df[["organization name", value]].iloc[:start], self.results_df], axis=0)
            df = df.drop(columns=[value])

        batch_prompts = []
        batch_indices = []

        for i, row in list(df.iterrows())[start:end]:
            name = row['organization name']
            website = row['website']
            prompt = prompt_template

            for arg in args:
                prompt = prompt.replace(f"${arg[0]}", row[arg[1]])

            batch_prompts.append((i, name, prompt))
            batch_indices.append(i)

            if len(batch_prompts) >= batch_size:
                await self.process_batch(batch_prompts)
                batch_prompts = []

        if batch_prompts:
            await self.process_batch(batch_prompts)
            self.results_df.to_csv("../output/current_results_df_prompting.csv")

        df = df.merge(self.results_df, on='organization name', how='left')
        return df

    async def process_batch(self, batch_prompts):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i, name, prompt in batch_prompts:
                tasks.append(self.fetch_result(session, i, name, prompt))
            await asyncio.gather(*tasks)

    async def fetch_result(self, session, i, name, prompt):
        failure_count = 0
        while True:
            try:
                print(f"******************************\nProcessing {i}: {name}")
                result = await x_gemini.ask(session, prompt)
                if result == "N/A": break  # explicit material

                text = re.sub(r"#|#\s+|_|\*", "", result).strip()

                self.results_df.loc[i] = [name, text]

                    
                break
            except Exception as e:
                print(failure_count)
                failure_count += 1
                if failure_count > 10:
                    break
                print(f"Error processing {i}, {name}: {e}")
                await asyncio.sleep(20)

prompting_class = prompting()


In [32]:
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, generated_description_prompt, args, "generated_description_llm")
    df.to_csv('../output/df_with_generated_description.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 0: Pika
******************************
Processing 1: Contextual AI
******************************
Processing 2: Sierra
******************************
Processing 3: Essential AI
******************************
Processing 4: Liquid AI
******************************
Processing 5: Lindy
******************************
Processing 6: LlamaIndex
******************************
Processing 7: MagicSchool AI
******************************
Processing 8: Norm AI
******************************
Processing 9: Patronus AI
******************************
Processing 10: Sixfold AI
******************************
Processing 11: Voltage Park
******************************
Processing 12: Elicit
******************************
Processing 13: Mercor
******************************
Processing 14: Guardrails AI
******************************
Processing 15: VectorShift
******************************
Processing 16: Portkey AI
******************************
Processing 17: MyShel

In [78]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace(" (two sentences)","").replace("/10","").replace("_","").replace("#","")
    pattern = r"^.*?\s*Description:?\s*(.*)Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["generated_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)
len(df)

2196

In [77]:
df = pd.read_csv("../output/df_with_generated_description.csv", index_col=0).reset_index(drop=True)
len(df)

2196

In [48]:
# Restart if cut in the middle of run!
# df = df.merge(prompting_class.results_df, on='organization name', how='left')

In [79]:
df = df.dropna(subset=["generated_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2194

In [80]:
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]  
args = [["company","organization name"],["website","website"], ["description","generated_description"]]

async def main(df):
    df = await prompting_class.iterate(df, parsed_description_prompt, args, "parsed_description_llm")
    df.to_csv('../output/df_with_parsed_description.csv', index=False)
    return df

df = asyncio.run(main(df))




******************************
Processing 0: Pika
******************************
Processing 1: Contextual AI
******************************
Processing 2: Sierra
******************************
Processing 3: Essential AI
******************************
Processing 4: Liquid AI
******************************
Processing 5: Lindy
******************************
Processing 6: LlamaIndex
******************************
Processing 7: MagicSchool AI
******************************
Processing 8: Norm AI
******************************
Processing 9: Patronus AI
******************************
Processing 10: Sixfold AI
******************************
Processing 11: Voltage Park
******************************
Processing 12: Elicit
******************************
Processing 13: Mercor
******************************
Processing 14: Guardrails AI
******************************
Processing 15: VectorShift
******************************
Processing 16: Portkey AI
******************************
Processing 17: MyShel

In [81]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text)  
    text = text.replace("_", "").replace("*", "").replace("#", "")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(.*?)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": 
            "Tasks/Jobs: " + tasks_jobs + "\n" + "Industry: " + industry + "\n" + "Customers: "+ customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print(text)
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["parsed_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [82]:
len(df)

2194

In [83]:
df = df.dropna(subset=["parsed_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2194

In [84]:
df.to_csv('../output/df_with_parsed_description.csv', index=False)

In [85]:
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_reasoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_reasoning","situation3", "situation3_conf_interval", "situation3_conf_interval_reasoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']
args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]


async def main(df):
    df = await prompting_class.iterate(df, examples_prompt, args, "examples_llm",1060)
    df.to_csv('../output/df_with_examples.csv', index=False)
    return df

df = asyncio.run(main(df))


******************************
Processing 0: Pika
******************************
Processing 1: Contextual AI
******************************
Processing 2: Sierra
******************************
Processing 3: Essential AI
******************************
Processing 4: Liquid AI
******************************
Processing 5: Lindy
******************************
Processing 6: LlamaIndex
******************************
Processing 7: MagicSchool AI
******************************
Processing 8: Norm AI
******************************
Processing 9: Patronus AI
******************************
Processing 10: Sixfold AI
******************************
Processing 11: Voltage Park
******************************
Processing 12: Elicit
******************************
Processing 13: Mercor
******************************
Processing 14: Guardrails AI
******************************
Processing 15: VectorShift
******************************
Processing 16: Portkey AI
******************************
Processing 17: MyShel

In [101]:

def extract_data(text):
    if pd.isnull(text): return {}
    # Normalize the text to ensure consistent whitespace and remove unwanted characters.
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("*", "").replace("/10", "").replace("_", "").replace("#", "").replace("- ","")
    vals = {}
    i = 1
    while i <=3:
        # Regex pattern adjusted to handle fractions in confidence intervals like '9/10'
        pattern = rf"Example\s+{i}:\s+(.*?)\s+ONET\s+JOB\s+automated\s+{i}:\s+(.*?)\s+ONET\s+JOB\s+{i}:\s+(.*?)\s+Confidence\s+Interval\s+{i}:\s+(\d+(?:/\d+)?)\s+Reasoning\s+{i}:\s+(.*?)(?=\s*Example\s+{i + 1}:|$)"
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            print(text+"\n\n")
            print(f"No matches found for Example {i}")  # Debug if no examples are found
            return {}
            break

        example_text, onet_job_automated, onet_job, confidence_interval, reasoning = match.groups()
        example_key = f"Example{i}"
        vals[example_key] = example_text.strip()
        vals[f"Job{i}"] = onet_job_automated.strip()
        vals[f"Job{i}_title"] = onet_job.strip()
        vals[f"situation{i}_conf_interval"] = confidence_interval.strip()
        vals[f"situation{i}_conf_interval_reasoning"] = reasoning.strip()

        i += 1  # Prepare to search for the next example
    return vals

for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])

results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["examples_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


OkamiAI Example Scenarios Company: OkamiAI Website: — Current Description: Unfortunately, I do not have access to the internet to gather information about OkamiAI or its products. Without a website or more information, I am unable to provide a product description. I will create examples based on hypothetical functionalities commonly found in AI-powered tools for different industries. Tasks/Jobs: AI-powered automation, data analysis, decision support. Industry: Could be applicable across various sectors like healthcare, finance, manufacturing, etc. Customers: Businesses of various sizes. Example 1: A customer service representative at a large retail company uses OkamiAI to automatically analyze customer emails and identify the nature of the issue, providing the representative with a suggested response and relevant information from company databases. ONET JOB automated 1: Customer Service Representatives that answer questions about products or services, handle complaints, and provide inf

In [103]:
df = df.dropna(subset=["Example1"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2191

In [104]:
df.to_csv('../output/df_with_examples.csv', index=False)

In [20]:
#RERUN STOPPED IN MIDDLE
# results_df = pd.read_csv("result.csv",index_col=0)
# df = pd.read_csv('../output/df_with_parsed_description.csv',index_col=0)
# prompting_class.set_current_results_df(results_df)
# df = df.merge(results_df, on='organization name', how='left')